In [5]:
import pickle
from datetime import datetime
import json

In [6]:
folder = 'data/comp1_2020_1'

In [7]:
data_inicio = datetime.strptime('30/11/2020 00:00:00', '%d/%m/%Y %H:%M:%S')
data_fim = datetime.strptime('06/03/2021 23:59:59', '%d/%m/%Y %H:%M:%S')

In [8]:
turmas_validas = OnlineClass.objects.filter(start_date__gte=data_inicio, 
                                            start_date__lte=data_fim).values_list('pk', flat=True)
turmas_validas

<QuerySet [36, 30, 25, 26, 38, 37, 54, 33, 34, 39, 27, 28, 29, 32, 31]>

In [51]:
# Separando chapters PLE
valid_chapters = Chapter.objects.filter(label__contains='PLE') | Chapter.objects.filter(label__contains='10')
valid_chapters.count()

9

In [52]:
# Remove professores e usuarios que já tenham utilizado o sistema em outra turma
professores = Professor.objects.all().values_list('user')
usuarios_tentativa_anterior = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas,
                                                     timestamp__lt=data_inicio).values_list('user').distinct()

In [53]:
# O que fazer com alunos repetentes?
# Vou separá-los por enquanto

### Get logs from classes with corrected assigned chapters

In [54]:
%%time
userlog = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas,
                                 problem__chapter__in=valid_chapters
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class',
                                         'problem__chapter')
print(userlog.count())

107876
CPU times: user 16.9 ms, sys: 2.49 ms, total: 19.4 ms
Wall time: 399 ms


In [55]:
students = list(set([log['user__id'] for log in userlog]))
problems = list(set([log['problem__id'] for log in userlog]))
classes = list(set([log['user__userprofile__user_class'] for log in userlog]))
chapters = list(set([log['problem__chapter'] for log in userlog]))

print(f"Total de alunos: {len(students)}")
print(f"Total de problemas: {len(problems)}")
print(f"Total de turmas: {len(classes)}")
print(f"Total de listas: {len(chapters)}")

Total de alunos: 427
Total de problemas: 50
Total de turmas: 15
Total de listas: 9


In [59]:
with open("%s/userlogs.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog), pklfile)

In [60]:
%%time
userlog_complete = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas, 
                                          problem__chapter__in=valid_chapters
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class',
                                         'solution', 'seconds_in_page', 'seconds_in_code', 
                                         'seconds_to_begin', 'problem__chapter')
print(userlog_complete.count())

with open("%s/userlogs_complete.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog_complete), pklfile)

107876
CPU times: user 635 ms, sys: 87.6 ms, total: 723 ms
Wall time: 7.98 s


In [61]:
# One row per test case
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values(
    "problem_id", "content", "header", "problem__testcase__content").order_by("content")

In [62]:
%%time
data = []
old_content = ""
for item in solutions:
    # If it is the same problem, keep adding test cases
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    # Otherwise, start new row
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]

CPU times: user 3.96 ms, sys: 1.33 ms, total: 5.29 ms
Wall time: 167 ms


In [11]:
%%time
# Transform it into dict to lookup for problem
data_dict = {}
for item in data:
    data_dict[item["problem_id"]] = item

CPU times: user 28 µs, sys: 19 µs, total: 47 µs
Wall time: 49.6 µs


In [ ]:
# Calculate test case percentage for each userlog
userlog_testcase = []
for item in userlog:
    tcs = data[userlog.problem_id]["test_case"]
    correct = 0
    # Calculate student response and solution for each test case for each log
    for tc in tcs:
        # Transform solution into Python function
        function_obj = compile(data[userlog.problem_id]["solution"], 'solution', 'exec')
        exec(function_obj)
        answer_solution = eval(item["function_call"])(*tc)
        
        # Transform student solution into Python function
        
        function_obj = compile(userlog.solution, 'solution', 'exec')
        exec(function_obj)
        answer_student = eval(item["function_call"])(*tc)

In [ ]:
for item in data:
    for tc in item["test_case"]:
        print(tc)
        # Transform solution into python function
        function_obj = compile(item["solution"], 'solution', 'exec')
        exec(function_obj)
        answer = eval(item["function_call"])(*tc)
        